In [1]:
import pandas as pd

file_path = r'C:\Users\HP\Documents\Projets\Projet1\data1.xlsx'

try:
    # Lire le fichier Excel dans un DataFrame Pandas
    # Nous ajoutons l'encodage 'utf-8' au cas où il y aurait des caractères spéciaux
    df_brut = pd.read_excel(file_path)
    
    # Afficher les premières lignes du DataFrame pour vérifier
    print("✅ Données brutes lues avec succès :")
    print(df_brut.head())
    
    # Afficher le type de données et le nombre de lignes
    print("\n--- Informations sur le DataFrame ---")
    df_brut.info()
    
    # Stocker le texte brut pour l'étape suivante (Extraction LLM)
    full_text_input = "\n\n".join(df_brut['Texte_Brut'].astype(str).tolist())
    print(f"\nLe texte brut total à analyser contient {len(full_text_input)} caractères.")
    
except FileNotFoundError:
    # Cette erreur signifie que le fichier n'est pas trouvé à cet endroit précis
    print(f"❌ ERREUR : Le fichier n'a pas été trouvé au chemin '{file_path}'.")
    print("Vérifiez l'orthographe du chemin et le nom du fichier ('Projet 1 data RO.xlsx').")
except KeyError:
    # Cette erreur signifie que les en-têtes de colonnes ne sont pas corrects
    print("❌ ERREUR : Le DataFrame ne contient pas la colonne 'Texte_Brut'. Vérifiez l'orthographe de l'en-tête de colonne dans votre fichier Excel.")
except Exception as e:
    print(f"❌ Une erreur inattendue est survenue : {e}")

✅ Données brutes lues avec succès :
  ID_Commande                 Texte_Brut (À extraire par le LLM)
0     ORD-A-1  La P-Alpha (48 kilos, 10 m³) doit être livrée....
1     ORD-B-2  Note urgente : Le colis M-Beta fait environ 30...
2     ORD-C-3  Pour le C-Gamma, il est à 42 kg et 12 m³. Sa v...
3     ORD-D-4  Le plus important est le G-Delta, qui pèse 36 ...
4     ORD-E-5  N'oublie pas S-Epsilon (36 kg, 9 m³). Valeur :...

--- Informations sur le DataFrame ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   ID_Commande                         10 non-null     object
 1   Texte_Brut (À extraire par le LLM)  10 non-null     object
dtypes: object(2)
memory usage: 292.0+ bytes
❌ ERREUR : Le DataFrame ne contient pas la colonne 'Texte_Brut'. Vérifiez l'orthographe de l'en-tête de colonne dans votre fichier Ex

In [4]:
# Installation des librairies
!pip install pandas langchain-openai pydantic python-dotenv

'DOSKEY' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.

[notice] A new release of pip is available: 25.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import pandas as pd
from pydantic import BaseModel, Field
from typing import List
from langchain_openai import ChatOpenAI
#from langchain.output_parsers import PydanticOutputParser
#from langchain.prompts import PromptTemplate
import os
from dotenv import load_dotenv

# --- 1. Charger la clé API ---
# Assurez-vous d'avoir un fichier .env contenant : OPENAI_API_KEY="votre_cle"
load_dotenv() 

# ----------------------------------------------------
# A. DÉFINITION DU SCHÉMA (Pydantic)
# ----------------------------------------------------
class Item(BaseModel):
    """Représente un colis ou un article à charger."""
    # Description précise pour aider le LLM à extraire la bonne information
    ID_Colis: str = Field(description="L'identifiant unique de l'article (ex: P-Alpha, M-Beta).")
    Poids_kg: int = Field(description="Le poids de l'article en kilogrammes. DOIT ÊTRE UN NOMBRE ENTIER.")
    Volume_m3: int = Field(description="Le volume de l'article en mètres cubes. DOIT ÊTRE UN NOMBRE ENTIER.")
    Priorite: int = Field(description="Niveau de priorité ou valeur à maximiser. DOIT ÊTRE UN NOMBRE ENTIER.")

class ExtractedData(BaseModel):
    """Conteneur pour la liste de tous les items extraits du texte."""
    items: List[Item] = Field(description="La liste complète de tous les articles trouvés dans la commande brute.")

print("✅ Schéma Pydantic défini.")

✅ Schéma Pydantic défini.


In [7]:
# Le chemin d'accès complet (absolu) de votre fichier
file_path = r'C:\Users\HP\Documents\Projets\Projet1\data1.xlsx' 
COLUMN_NAME = 'Texte_Brut (À extraire par le LLM)' # Nom exact de la colonne

try:
    df_brut = pd.read_excel(file_path)
    
    # Concaténer tout le texte brut en une seule grande chaîne.
    # Chaque commande sera séparée par un double saut de ligne pour aider le LLM.
    full_text_input = "\n\n".join(df_brut[COLUMN_NAME].astype(str).tolist())
    
    print(f"✅ Fichier lu et {len(df_brut)} lignes de commandes concaténées en une seule entrée pour le LLM.")
    # Affiche un aperçu pour confirmation
    print("\n--- Début de l'input LLM ---")
    print(full_text_input[:300] + "...") 
    
except Exception as e:
    print(f"❌ ERREUR lors de la lecture ou de la préparation de l'input : {e}")
    # Arrête l'exécution si les données n'ont pas pu être chargées
    raise

✅ Fichier lu et 10 lignes de commandes concaténées en une seule entrée pour le LLM.

--- Début de l'input LLM ---
La P-Alpha (48 kilos, 10 m³) doit être livrée. Priorité (valeur) : 10.

Note urgente : Le colis M-Beta fait environ 30kg et seulement 8 mètres cubes. Il a une forte valeur : 30.

Pour le C-Gamma, il est à 42 kg et 12 m³. Sa valeur est de 25.

Le plus important est le G-Delta, qui pèse 36 kg et fait ...


In [9]:
!pip install langchain langchain-openai openai

'DOSKEY' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.

[notice] A new release of pip is available: 25.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
from dotenv import load_dotenv
import os

# Remplace "chemin/vers/.env" par le chemin réel
load_dotenv(dotenv_path=r'C:\Users\HP\Documents\Projets\Projet1')

print(os.getenv("OPENAI_API_KEY"))  # Devrait afficher ta clé

None


In [17]:
from dotenv import load_dotenv
import os

load_dotenv()  # charge le .env
api_key = os.getenv("OPENAI_API_KEY")
assert api_key is not None, "Clé API non trouvée ! Vérifie ton .env"
print("Clé API OK :", api_key[:4] + "…")  # affiche seulement les 4 premiers caractères


Clé API OK : sk-p…


In [18]:
# ----------------------------------------------------
# B. PRÉPARATION DES OUTILS LANGCHAIN (suite)
# ----------------------------------------------------
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate

# 1. Configuration du LLM (si non fait dans la Cellule 2)
# Nous utilisons gpt-3.5-turbo qui est rapide et efficace pour cette tâche.
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0 ,openai_api_key=os.getenv("OPENAI_API_KEY")) 

# 2. Le Parser Pydantic pour formater la sortie
parser = PydanticOutputParser(pydantic_object=ExtractedData)

# 3. Le Prompt Engineering (Instruction donnée au LLM)
format_instructions = parser.get_format_instructions()
prompt_template = """
Tu es un expert en traitement de commandes logistiques. Ta tâche est d'analyser le TEXTE DE COMMANDE BRUT ci-dessous 
et d'en extraire TOUTES les informations d'articles, en respectant rigoureusement le FORMAT JSON STICT. 
Pour chaque item trouvé, extrait son ID, son poids en kg, son volume en m3, et sa priorité (valeur). 
Convertis TOUTES les valeurs numériques en nombres ENTIERS (int).

TEXTE DE COMMANDE BRUT:
---
{commande_text}
---

{format_instructions}
"""
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["commande_text"],
    partial_variables={"format_instructions": format_instructions}
)

# ----------------------------------------------------
# C. EXÉCUTION
# ----------------------------------------------------

# La chaîne de LangChain : Prompt -> LLM -> Parser
chain = prompt | llm | parser

print("🚀 Démarrage de l'extraction LLM... (cela peut prendre quelques secondes)")

try:
    # Exécuter l'extraction sur l'ensemble du texte concaténé
    extracted_data_pydantic = chain.invoke({"commande_text": full_text_input})
    
    # Transformer le modèle Pydantic en DataFrame Pandas
    extracted_items = [item.model_dump() for item in extracted_data_pydantic.items]
    df_clean = pd.DataFrame(extracted_items)
    
    # Enregistrer les données propres
    df_clean.to_csv('donnees_propres_RO.csv', index=False)
    
    print("\n✅ Extraction terminée avec succès!")
    print(f"Données structurées enregistrées dans 'donnees_propres_RO.csv'")
    print("\n--- Aperçu des données extraites (Prêtes pour la RO) ---")
    print(df_clean)

except Exception as e:
    print(f"\n❌ ERREUR lors de l'extraction. Vérifiez votre clé API ou si le LLM a renvoyé un format incorrect : {e}")

🚀 Démarrage de l'extraction LLM... (cela peut prendre quelques secondes)

❌ ERREUR lors de l'extraction. Vérifiez votre clé API ou si le LLM a renvoyé un format incorrect : Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


In [14]:
from dotenv import load_dotenv
import os

load_dotenv()  # charge le .env
api_key = os.getenv("OPENAI_API_KEY")
print("Clé API :", api_key)


Clé API : None
